In [1]:
import pandas as pd
import os
import numpy as np
import epiweeks
import datetime
import requests
import zipfile
import csv
from contextlib import ExitStack
import progressbar as pbar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
import warnings
from itertools import product
warnings.filterwarnings('ignore')
from openpyxl import load_workbook

In [ ]:
today = datetime.date.today().strftime("%d-%m-%Y")
data_dir_path = f'data/World/{today}'
result_dir_path = f'result/World/{today}'
if not os.path.exists(f'{result_dir_path}'):
    os.mkdir(f'{result_dir_path}')
if not os.path.exists(f'{result_dir_path}/JHU'):
    os.mkdir(f'{result_dir_path}/JHU')
with open(f'{data_dir_path}/eurostat_countries.json', 'r') as file:
    countries = json.load(file)

In [3]:
format_label = pbar.FormatCustomText('Country: %(country)s Running: %(running)s   ', dict(country='start', running='start'))
widgets = [pbar.ETA(), ' ', pbar.Timer(), ' ', pbar.Percentage(format=' (%(percentage)3d%%) '), '  ', format_label]
bar = pbar.ProgressBar(widgets=widgets, maxval=len(countries))
bar.start()

age_group_translator = {'Y_LT5': '00-04', 'Y5-9': '05-09',
                        'Y10-14': '10-14', 'Y15-19': '15-19',
                        'Y20-24': '20-24', 'Y25-29': '25-29',
                        'Y30-34': '30-34', 'Y35-39': '35-39',
                        'Y40-44': '40-44', 'Y45-49': '45-49',
                        'Y50-54': '50-54', 'Y55-59': '55-59',
                        'Y60-64': '60-64', 'Y65-69': '65-69',
                        'Y70-74': '70-74', 'Y75-79': '75-79',
                        'Y80-84': '80-84', 'Y_GE85': '85+',
                        'Y85-89': '85-89', 'Y_GE90': '90+',}

with open(f'{data_dir_path}/ifr_5_years_age_group.json', 'r') as file:
    IFRs = json.load(file)

df_ratio = {}
for country_index, (country, regions) in enumerate(countries.items()):
    if country != 'Germany':
        continue
    format_label.update_mapping(country=country)
    bar.update()
    
    # Load cases
    format_label.update_mapping(running='Loading cases' + ' ' * 30)
    bar.update()
    jhd_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    jhd_cases = jhd_cases[jhd_cases['Country/Region'] == country].fillna(country)
    jhd_cases = jhd_cases[jhd_cases['Province/State'] == country].drop(columns=['Province/State', 'Country/Region', 'Lat', 'Long']).T
    jhd_cases.index = pd.to_datetime(jhd_cases.index)
    jhd_cases.columns = ['value']
    jhd_cases['value'] = [0, *(jhd_cases.value[1:].to_numpy() - jhd_cases.value[:-1].to_numpy())]
    cases = jhd_cases.reset_index().rename(columns={'index': 'date'}).reindex(columns=['date', 'location', 'location_name', 'Age group', 'value']).fillna({'location': country, 'location_name': countries[country], 'Age group': 'TOTAL'})
    
    # Load deaths
    format_label.update_mapping(running='Loading deaths' + ' ' * 30)
    bar.update()
    jhd_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
    jhd_deaths = jhd_deaths[jhd_deaths['Country/Region'] == country].fillna(country)
    jhd_deaths = jhd_deaths[jhd_deaths['Province/State'] == country].drop(columns=['Province/State', 'Country/Region', 'Lat', 'Long']).T
    jhd_deaths.index = pd.to_datetime(jhd_deaths.index)
    jhd_deaths.columns = ['value']
    jhd_deaths['value'] = [0, *(jhd_deaths.value[1:].to_numpy() - jhd_deaths.value[:-1].to_numpy())]
    deaths = jhd_deaths.reset_index().rename(columns={'index': 'date'}).reindex(columns=['date', 'location', 'location_name', 'Age group', 'value']).fillna({'location': country, 'location_name': countries[country], 'Age group': 'TOTAL'})
    

    # Load population
    format_label.update_mapping(running='Loading population' + ' ' * 30)
    bar.update()
    population = pd.read_csv(f'{data_dir_path}/{country}/old_population.csv')
    population = population[~population['Age group'].isin(['UNK', 'Y_OPEN'])]
    population['Age group'] = population['Age group'].replace({'TOTAL': 'YTOTAL', 'Y_LT1': 'Y0'}).apply(lambda x: x[1:])
    population_yearly = population[~population['Age group'].isin(['TOTAL', '0'])]
    for start, stop in zip(range(0, 96, 5), range(5, 101, 5)): 
        population = population.append(population[population['Age group'].isin([str(i) for i in range(start, stop)])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
        population = population[~population['Age group'].isin([str(i) for i in range(start, stop)])]
        population['Age group'] = population['Age group'].fillna(f'{start:02d}-{stop-1:02d}')
    population = population.append(population[population['Age group'].isin(['85-89', '90-94', '95-99'])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
    population = population[~population['Age group'].isin(['85-89', '90-94', '95-99'])]
    population['Age group'] = population['Age group'].fillna(f'85+')
    
    
    if country == 'United Kingdom':
        population = population[population['Year'] == 2019].reset_index(drop=True)
    else:
        population = population[population['Year'] == 2020].reset_index(drop=True)
    new_df = pd.DataFrame({'Age group': population['Age group'].unique()})
    for i, region in enumerate([regions]):
        new_df.insert(i + 1, region, list(population[np.logical_and(population['Region'] == region, population['Sex'] == 'T')]['Value']), True)
    new_df = new_df[~new_df['Age group'].isin(['UNK', 'Y_GE75', 'Y_GE80'])].sort_values('Age group').reset_index(drop=True)
    population = new_df.copy()
    
    if country == 'United Kingdom':
        population_yearly = population_yearly[population_yearly['Year'] == 2019].reset_index(drop=True)
    else:
        population_yearly = population_yearly[population_yearly['Year'] == 2020].reset_index(drop=True)
    new_df = pd.DataFrame({'Age group': population_yearly['Age group'].unique()})
    for i, region in enumerate([regions]):
        new_df.insert(i + 1, region, list(population_yearly[np.logical_and(population_yearly['Region'] == region, population_yearly['Sex'] == 'T')]['Value']), True)
    new_df = new_df[~new_df['Age group'].isin(['UNK', 'Y_GE75', 'Y_GE80'])].sort_values('Age group').reset_index(drop=True)
    population
    
    bar.update()
    
    # Load and prepare total deaths
    format_label.update_mapping(running='Loading total deaths' + ' ' * 30)
    bar.update()
    total_deaths = pd.read_csv(f'{data_dir_path}/{country}/old_deaths.csv')
    total_deaths = total_deaths[total_deaths['Region'] == regions]
    total_deaths = total_deaths[total_deaths['Sex'] == 'T'].reset_index(drop=True)
    total_deaths = total_deaths[total_deaths['Year'].apply(lambda x: x[:4]) >= str(2015)].reset_index(drop=True)
    new_df = pd.DataFrame(columns=['Year', 'Region', 'Age group'] + list(np.arange(1, 54)))
    new_df = new_df.append(total_deaths[total_deaths['Sex'] == 'T'])
    new_df['Year'] = new_df['Year'].apply(lambda x: x[:4])
    new_df = new_df.drop(columns='Sex', axis=1)
    new_df = new_df.drop(columns='Value', axis=1)
    new_df = new_df.drop_duplicates()
    bar.update()
    
    for i in range(len(total_deaths)):
        region = total_deaths.iloc[i]['Region']
        age_group = total_deaths.iloc[i]['Age group']
        year = total_deaths.iloc[i]['Year'][:4]
        week = int(total_deaths.iloc[i]['Year'][6:])
        x = np.logical_and(new_df['Region'] == region, new_df['Year'] == year)
        x = np.logical_and(x, new_df['Year'] >= str(2015))
        x = np.logical_and(x, new_df['Age group'] == age_group)
        new_df.loc[x, week] = total_deaths.iloc[i]['Value']
        bar.update()
    total_deaths = new_df[new_df['Year'] >= str(2015)]
    total_deaths = total_deaths.drop(total_deaths.loc[total_deaths['Age group']=='UNK'].index)
    total_deaths = total_deaths.drop(total_deaths.loc[total_deaths['Age group']=='Y_GE75'].index)
    total_deaths = total_deaths.drop(total_deaths.loc[total_deaths['Age group']=='Y_GE80'].index).fillna(0)
    total_deaths_5_years_age_group = total_deaths.copy()
    total_deaths.replace(age_group_translator, inplace=True)
    total_deaths = total_deaths.append(total_deaths[total_deaths['Age group'].isin(['85-89', '90+'])].groupby(['Year', 'Region']).sum().reset_index())
    total_deaths = total_deaths.fillna({'Age group': '85+'})
    total_deaths = total_deaths[~total_deaths['Age group'].isin(['85-89', '90+'])].reset_index(drop=True)
    bar.update()
    
    # Load and prepare prior population
    format_label.update_mapping(running='Loading prior population' + ' ' * 30)
    bar.update()
    old_population = pd.read_csv(f'{data_dir_path}/{country}/old_population.csv')

    old_population = old_population[~old_population['Age group'].isin(['UNK', 'Y_GE75', 'Y_GE80', 'Y_OPEN'])]

    old_population['Age group'] = old_population['Age group'].replace({'TOTAL': 'YTOTAL', 'Y_LT1': 'Y0'}).apply(lambda x: x[1:])
    for start, stop in zip(range(0, 96, 5), range(5, 101, 5)): 
        old_population = old_population.append(old_population[old_population['Age group'].isin([str(i) for i in range(start, stop)])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
        old_population = old_population[~old_population['Age group'].isin([str(i) for i in range(start, stop)])]
        old_population['Age group'] = old_population['Age group'].fillna(f'{start:02d}-{stop-1:02d}')
    old_population = old_population.append(old_population[old_population['Age group'].isin(['85-89', '90-94', '95-99'])].groupby(['Year', 'Region', 'Sex']).sum().reset_index())
    old_population = old_population[~old_population['Age group'].isin(['85-89', '90-94', '95-99'])]
    old_population['Age group'] = old_population['Age group'].fillna(f'85+')

    old_population = old_population[np.logical_and(old_population['Year'] >= 2015, old_population['Year'] < 2020)].reset_index(drop=True)
    prior_populations = dict()

    for year in range(2015, 2020):
        df = pd.DataFrame(columns=['Age group'] + [regions])
        df['Age group'] = old_population['Age group'].unique()
        for region in [regions]:
            condition = np.logical_and(old_population['Region'] == region, old_population['Year'] == year)
            condition = np.logical_and(condition, old_population['Sex'] == 'T')
            if len(old_population[condition]['Value']) < len(df['Age group']):
                df.loc[:, region] = 0
            else:
                df.loc[:, region] = list(old_population[condition]['Value'])
            bar.update()
        df = df.drop(df.loc[df['Age group']=='UNK'].index)
        df = df.drop(df.loc[df['Age group']=='Y_GE75'].index)
        df = df.drop(df.loc[df['Age group']=='Y_GE80'].index)
        df['Age group'].replace(age_group_translator, inplace=True)
        prior_populations[year] = df.sort_values(by='Age group').reset_index().drop('index', axis=1)
    bar.update()
    
#     # Prepare regions set
#     cases_regions_set = set(cases['location_name'].unique())
#     deaths_regions_set = set(deaths['location_name'].unique())
#     population_regions_set = set(population.columns)
#     total_deaths_regions_set = set(total_deaths['Region'].unique())
#     prior_populations_regions_set = set(prior_populations[2015].columns)
#     regions_set = cases_regions_set
#     regions_set = regions_set.intersection(deaths_regions_set)
#     regions_set = regions_set.intersection(population_regions_set)
#     regions_set = regions_set.intersection(total_deaths_regions_set)
#     regions_set = regions_set.intersection(prior_populations_regions_set)
#     df = pd.DataFrame(columns= ['Year', 'Region', 'Age group'] + list(range(1, 53+1)))
    
#     for region in regions_set:
#         for year in total_deaths['Year'].unique():
#             new_df = total_deaths[np.logical_and(total_deaths['Region'] == region, total_deaths['Year'] == year)].iloc[:, 2:].set_index('Age group')
#             new_df = new_df.reset_index().rename(columns = {'index': 'Age group'})
#             df = df.append(new_df)
#             df['Year'] = df['Year'].fillna(year)
#             df['Region'] = df['Region'].fillna(region)
#     total_deaths = df
    
#     for region in regions_set:
#         # Prepare dir
#         format_label.update_mapping(running='Prepare dir' + ' ' * 30)
#         bar.update()
#         if not os.path.exists(f'{result_dir_path}/{country}'):
#             os.mkdir(f'{result_dir_path}/{country}')
#         region = region.replace('*', '')
#         if not os.path.exists(f'{result_dir_path}/{country}/{region}'):
#             os.mkdir(f'{result_dir_path}/{country}/{region}')  
            
#         # Simulation
#         format_label.update_mapping(running='Simulation' + ' ' * 30)
#         bar.update()
#         daily_regional_deaths, weekly_regional_deaths = prepare_weekly_covid_deaths(deaths, region)
#         weekly_regional_cases = prepare_weekly_covid_cases(cases, region)
#         weekly_regional_cases = weekly_regional_cases.append(pd.DataFrame(weekly_regional_cases.sum(axis=0), columns=['TOTAL']).T)
#         weekly_regional_deaths = weekly_regional_deaths.append(pd.DataFrame(weekly_regional_deaths.sum(axis=0), columns=['TOTAL']).T)
#         correction_due_to_aging = calculate_correction_due_to_aging(prior_populations, population, region)
#         max_deaths = calculate_reference_max_deaths(region, correction_due_to_aging, total_deaths)
#         overall_deaths = calculate_overall_covid_deaths(total_deaths, region).apply(pd.to_numeric)
        
#         overall_minus_max = np.maximum(0, overall_deaths - max_deaths).loc[:, :weekly_regional_deaths.columns[-1]]
#         additional_deaths_not_attributed_to_covid = np.maximum(0, overall_minus_max[weekly_regional_deaths.columns] - weekly_regional_deaths).fillna(0)
        
#         # Plot results
#         format_label.update_mapping(country=country, running='Ploting result: weekly covid deaths' + ' ' * 30)
#         bar.update()
#         plot_weekly_deaths()
        
#         format_label.update_mapping(country=country, running='Ploting result: weekly covid cases' + ' ' * 30)
#         bar.update()
#         plot_weekly_cases()
        
#         format_label.update_mapping(country=country, running='Ploting result: cumulative covid deaths' + ' ' * 30)
#         bar.update()
#         plot_cumulative_deaths()
#         format_label.update_mapping(country=country, running='Ploting result: darkfigure' + ' ' * 30)
#         bar.update()
#         if country == 'United Kingdom':
#             weekly_regional_cases = weekly_regional_cases.iloc[:, :-8]
#         window_list = [1, 3, 5]
#         gamma_list = [0.0, 0.8, 1.0]

#         for gamma, window in product(gamma_list, window_list):
#             format_label.update_mapping(country=country, running=f'Ploting result: darkfigure {gamma=}, {window=}' + ' ' * 30)
#             bar.update()
            
            
#             age_group = 'TOTAL'
#             fig, ax = plt.subplots(3, 2, figsize=(30,20))

#             weekly_regional_deaths_window = weekly_regional_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()
#             overall_minus_max_window = np.maximum(0, overall_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean() - max_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()).loc[:, :weekly_regional_deaths.columns[-1]]
#             additional_deaths_not_attributed_to_covid_window = np.maximum(0, overall_minus_max_window - weekly_regional_deaths_window).fillna(0)
#             corrected_covid_deaths_window = additional_deaths_not_attributed_to_covid_window * gamma + weekly_regional_deaths_window

#             overall_minus_max = np.maximum(0, overall_deaths - max_deaths).loc[:, weekly_regional_deaths.columns]
#             additional_deaths_not_attributed_to_covid = np.maximum(0, overall_minus_max - weekly_regional_deaths).fillna(0)
#             corrected_covid_deaths = additional_deaths_not_attributed_to_covid * gamma + weekly_regional_deaths

#             additional_deaths_not_attributed_to_covid_window.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[0, 0], label=f'excess {gamma=:.2f}', style='b')
#             additional_deaths_not_attributed_to_covid.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[0, 1], label=f'excess {gamma=:.2f}', style='b')


#             weekly_regional_deaths_window.loc[age_group].T.plot(ax=ax[1, 0], style='g', label='covid')
#             corrected_covid_deaths_window.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[1, 0], label=f'covid + excess {gamma=:.2f}', style='b')

#             weekly_regional_deaths.loc[age_group].T.plot(ax=ax[1, 1], style='g', label='covid')
#             corrected_covid_deaths.loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[1, 1], label=f'covid + excess {gamma=:.2f}', style='b')

#             weekly_regional_deaths_window.cumsum(axis=1).loc[age_group].T.plot(ax=ax[2, 0], label=f'covid', style='g')
#             corrected_covid_deaths_window.cumsum(axis=1).loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[2, 0], label=f'covid + excess {gamma=:.2f}', style='b')
#             corrected_covid_deaths.cumsum(axis=1).loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[2, 0], label=f'covid + excess (without mean) {gamma=:.2f}', style='--r')

#             weekly_regional_deaths.cumsum(axis=1).loc[age_group].T.plot(ax=ax[2, 1], label='covid', style='g')
#             corrected_covid_deaths.cumsum(axis=1).loc[age_group, weekly_regional_deaths.columns].T.plot(ax=ax[2, 1], label=f'covid + excess {gamma=:.2f}', style='b')

#             ax[0, 1].legend()
#             ax[0, 0].legend()
#             ax[1, 1].legend()
#             ax[1, 0].legend()
#             ax[2, 1].legend()
#             ax[2, 0].legend()

#             ax[0, 0].set_ylim([0, round(np.max(np.max(overall_minus_max)), -2) * 1.1])
#             ax[0, 1].set_ylim([0, round(np.max(np.max(overall_minus_max)), -2) * 1.1])
#             ax[1, 0].set_ylim([0, round(np.max(np.max(weekly_regional_deaths)), -2) * 1.1])
#             ax[1, 1].set_ylim([0, round(np.max(np.max(weekly_regional_deaths)), -2) * 1.1])
#             ax[2, 0].set_ylim([0, round(np.max(np.max(corrected_covid_deaths.cumsum(axis=1))), -2) * 1.1])
#             ax[2, 1].set_ylim([0, round(np.max(np.max(corrected_covid_deaths.cumsum(axis=1))), -2) * 1.1])

#             ax[0, 0].set_title(f'excess deaths with mean {window=}')
#             ax[0, 1].set_title('excess deahts without mean')
#             ax[1, 0].set_title(f'covid deaths with mean {window=} (weekly)')
#             ax[1, 1].set_title('covid deaths without mean (weekly)')
#             ax[2, 0].set_title(f'covid deaths with mean {window=} (cumulative)')
#             ax[2, 1].set_title('covid deaths without mean (cumulative)')

#             ax[0, 1].grid()
#             ax[0, 0].grid()
#             ax[1, 1].grid()
#             ax[1, 0].grid()
#             ax[2, 1].grid()
#             ax[2, 0].grid()
            
#             ax[0, 0].set_xlabel('week')
#             ax[0, 1].set_xlabel('week')
#             ax[1, 0].set_xlabel('week')
#             ax[1, 1].set_xlabel('week')
#             ax[2, 0].set_xlabel('week')
#             ax[2, 1].set_xlabel('week')
            
#             ax[0, 0].set_ylabel('deaths')
#             ax[0, 1].set_ylabel('deaths')
#             ax[1, 0].set_ylabel('deaths')
#             ax[1, 1].set_ylabel('deaths')
#             ax[2, 0].set_ylabel('deaths')
#             ax[2, 1].set_ylabel('deaths')
#             if not os.path.exists(f'{result_dir_path}/{country}/{region}/Window comparation/'):
#                 os.mkdir(f'{result_dir_path}/{country}/{region}/Window comparation/')
#             plt.savefig(f'{result_dir_path}/{country}/{region}/Window comparation/window={window}_gamma={gamma}.png')
#             plt.close('all')
#             bar.update()
            
#             cases_based_on_ifr = corrected_covid_deaths_window.copy().cumsum(axis=1)
#             df = weekly_regional_cases.rolling(window=window, axis=1, min_periods=1, center=True).mean().cumsum(axis=1)

#             for age_group, ifr in IFRs[country].items():
#                 if age_group in ['45-64', '50-64']:
#                     continue
#                 if age_group in ['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49']:
#                     df_60_64 = corrected_covid_deaths_window.copy().cumsum(axis=1).loc[['50-54', '55-59', '60-64'], :].sum().apply(lambda x: 100*x/IFRs[country]['50-64'])
#                     df.loc[age_group, :] = df_60_64 / df.loc[['50-54', '55-59', '60-64'], :].sum()
#                     df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
#                     df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))
#                     cases_based_on_ifr.loc[age_group, :] = (weekly_regional_cases.rolling(window=window, axis=1, min_periods=1, center=True).mean().loc[age_group, :] * df.loc[age_group, :]).cumsum()
#                 else:                                                                                                            
#                     cases_based_on_ifr.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :].apply(lambda x: 100*x/ifr)
#                     df.loc[age_group, :] = cases_based_on_ifr.loc[age_group, :] / df.loc[age_group, :]
#                     df.loc[age_group, :] = df.loc[age_group, :].fillna(0)
#                     df.loc[age_group, :] = df.loc[age_group, :].apply(lambda x: np.maximum(1.0, x))
                
#                 df_lim = 0.0
#                 fig, ax = plt.subplots(1, 2, figsize=(30, 10))
#                 cases_based_on_ifr.mul(population.loc[:, ['Age group', region]].set_index('Age group').apply(lambda x: 1/x).iloc[:, 0], axis=0).loc[age_group, weekly_regional_cases.columns].plot(ax=ax[0], label='based on ifr (gamma= 1.0)')
#                 weekly_regional_cases.mul(population.loc[:, ['Age group', region]].set_index('Age group').apply(lambda x: 1/x).iloc[:, 0], axis=0).loc[age_group, :].cumsum().plot(ax=ax[0], label='reported cases')
#                 ax[0].set_title(f'Covid cases based on IFR')
#                 ax[0].set_xlabel('week')
#                 ax[0].set_ylabel('cases / population [%]')
#                 ax[0].legend()
#                 df.loc[age_group, :].plot(ax=ax[1])
#                 ax[1].set_title(f'Darkfigure based on IFR')
#                 ax[1].set_xlabel('week')
#                 ax[1].set_ylabel('darkfigure')
#                 ax[0].grid()
#                 ax[1].grid()
#                 df_lim = np.max([df_lim, np.max(df.loc[age_group, :])])
#                 bar.update()
#                 if not os.path.exists(f'{result_dir_path}/{country}/{region}/Darkfigure/'):
#                     os.mkdir(f'{result_dir_path}/{country}/{region}/Darkfigure/')
#                 plt.savefig(f'{result_dir_path}/{country}/{region}/Darkfigure/window={window}_age_group={age_group}_gamma={gamma}.png')
#             plt.close('all')
#             bar.update()

#         format_label.update_mapping(country=country, running=f'Ploting result: Death factor' + ' ' * 30)
#         bar.update()
        
#         df_ratio[country] = pd.DataFrame()
#         for window in window_list:
#             weekly_regional_deaths_window = weekly_regional_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()
#             overall_minus_max_window = np.maximum(0, overall_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean() - max_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()).loc[:, :weekly_regional_deaths.columns[-1]]
#             additional_deaths_not_attributed_to_covid_window = np.maximum(0, overall_minus_max_window - weekly_regional_deaths_window).fillna(0)

#             week = '2021W22' # 31.05.2021 - 6.06.2021
#             weekly_regional_deaths_window = weekly_regional_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()
#             overall_minus_max_window = np.maximum(0, overall_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean() - max_deaths.rolling(window=window, axis=1, min_periods=1, center=True).mean()).loc[:, :weekly_regional_deaths.columns[-1]]
#             additional_deaths_not_attributed_to_covid_window = np.maximum(0, overall_minus_max_window - weekly_regional_deaths_window).fillna(0)

#             x1 = (additional_deaths_not_attributed_to_covid_window * 1.0 + weekly_regional_deaths_window).cumsum(axis=1)[week]
#             x0 = (additional_deaths_not_attributed_to_covid_window * 0.0 + weekly_regional_deaths_window).cumsum(axis=1)[week]
#             df_ratio[country][f'Ratio (covid deaths gamma=1 to population) {window=} [permil]'] = pd.DataFrame(x1.to_numpy() / population[region].to_numpy() * 1000, index=x1.index)
#             df_ratio[country][f'Ratio (covid deaths gamma=0 to population) {window=} [permil]'] = pd.DataFrame(x0.to_numpy() / population[region].to_numpy() * 1000, index=x0.index)
#             df_ratio[country][f'Ratio (covid deaths gamma=1 to gamma=0) {window=}'] = x1 / x0
            
            
            
#             fig, axs = plt.subplots(1, 2, figsize=(30,10))
#             for gamma in gamma_list:
#                 corrected_covid_deaths_window = additional_deaths_not_attributed_to_covid_window * gamma + weekly_regional_deaths_window
#                 corrected_covid_deaths_window.iloc[:-1].sum().cumsum().plot(ax=axs[0], label=f'gamma = {gamma}')
#                 corrected_covid_deaths_window.loc['TOTAL'].cumsum().plot(ax=axs[1], label=f'gamma = {gamma}')

#             axs[0].legend()
#             axs[0].set_xlabel('week')
#             axs[0].set_ylabel('deaths')
#             axs[0].legend()
#             axs[0].grid()
#             axs[0].set_title('Total corrected covid deaths by age group')
#             axs[0].set_ylim([0, round(np.max(np.max(corrected_covid_deaths_window.cumsum(axis=1))), -2) * 1.1])

#             axs[1].legend()
#             axs[1].set_xlabel('week')
#             axs[1].set_ylabel('deaths')
#             axs[1].grid()
#             axs[1].set_title('Total corrected covid deaths by total')
#             axs[1].set_ylim([0, round(np.max(np.max(corrected_covid_deaths_window.cumsum(axis=1))), -2) * 1.1])

#             if not os.path.exists(f'{result_dir_path}/{country}/{region}/Death factor/'):
#                 os.mkdir(f'{result_dir_path}/{country}/{region}/Death factor/')
#             plt.savefig(f'{result_dir_path}/{country}/{region}/Death factor/window={window}.png')
#             plt.close('all')
#             bar.update()
        
        
        
#     bar.update(country_index)
# df_ratio = pd.concat(df_ratio, axis=1)
# df_ratio.to_excel(f'{result_dir_path}/Ratio.xlsx', sheet_name=country)
bar.finish()
print('Simulation done')

Time:  0:00:06 Elapsed Time: 0:00:06  (100%)   Country: Germany Running: Loading prior population                                 


Simulation done
